In [2]:
#Abstract

In [ ]:
#Objectives

In [ ]:
#Part 1

In [ ]:
#Part 2

<center><b>Serial Time</b></center>

|Iteration|Time(s)|
|---------|:------:|
|1|47.557|
|2|47.331|
|3|47.416|
|||
|Mean|47.434|
|Std|0.114|


<center><b>Parallel Time</b></center>

|Iteration|Time(s)|
|---------|:------:|
|1|23.545|
|2|23.637|
|3|23.509|
|||
|Mean|23.564|
|Std|0.066|

<center>Performed timing for at least 3 iterations and until a standard deviation less than 1% of the mean was achieved </center>